# INFO 2950 PHASE 4

## Part 1: Introduction

#### Background Information
Drugs heavily impact the personality traits and characteristics of an individual, especially based on the frequency of their useage and of what drug. In this project, we aim to explore whether we can predict a person's drug use based on their personality traits. To do so, we would explore potentional correlations between specific personality traits and drug use,looking at: neuroticism, extraversion, openness to experience, agreeableness, conscientiousness, impulsivity, and sensation seeking. We will train a multivariable regression to see if we can reliably predict a person's drug use based on a person's personality traits. It is important to recognize hightened or unusual personality traits, especially in individuals who could be exposed to life-threatening drugs or unsafe situations of usage. We hope this evaluation will be a resource for readers to better understand effects of drug usage and what traits are heightened after using a certain drug. 

#### Research Questions
- Can we reliably predict a person's drug use based off a person's personality traits? 
- What types of drugs are more likely to be used by certain personalities?

Part 2: Data Description

-What are the observations (rows) and the attributes (columns)?
- Why was this dataset created?
- Who funded the creation of the dataset?
- What processes might have influenced what data was observed and recorded and
what was not?
- What preprocessing was done, and how did the data come to be in the form that
you are using?
- If people are involved, were they aware of the data collection and if so, what
purpose did they expect the data to be used for?
- Where can your raw source data be found, if applicable? Provide a link to the raw
data (hosted on Github, in a Cornell Google Drive or Cornell Box)

Part 3: Preregistration Statement

Part 4: Data Analysis

Part 5: Evaluation of Significance

Part 6: Conclusions

Part 7: Limitations

Part 8: Acknowledgements and Bibliography